# Dog Breeds Kaggle Competition -- fast.ai

## Setup

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

ImportError: No module named 'fastai'

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fhttp://localhost:8888/notebooks/DogBreedsKaggle.ipynb#astai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from 

In [ ]:
torch.cuda.set_device(1)

In [ ]:
PATH = "data/dogbreed/"
image_size=224
arch=resnext101_64
batch_size=32

In [ ]:
label_csv = f'{PATH}labels.csv'
num_labels = len(list(open(label_csv))-1

In [ ]:
# create a set of validation indices (i.e. indices not used for training -- by default it gets 20% of total for validation)
val_idxs = get_cv_idxs(num_labels)

## Data Exploration

In [ ]:
!ls {PATH}

In [ ]:
label_df = pd.read_csv(label_csv)

In [ ]:
label_df.head()

In [ ]:
label_df.pivot_table(index='breed', aggfunc=len).sort_values('id', ascending=False)

In [ ]:
# Use data augmentatioin on training images
tfms = tfms_from_model(arch, image_size, aug_tfms=transforms_side_on, max_zoom=1.1)

In [ ]:
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                    val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=batch_size)

In [ ]:
fn = PATH+data.trn_ds.fnames[0]; fn

In [ ]:
img = PIL.Image.open(fn); img

In [ ]:
img.size

In [ ]:
size_d = {k: PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames}

In [ ]:
row_size,col_size = list(zip(*size_d.values()))
row_size = np.array(row_size); col_size = np.array(col_size)

In [ ]:
row_size[:5]

In [ ]:
plt.hist(row_size)

In [ ]:
# plot histogram of row_sizes less than 1000
plt.hist(row_size[row_size<1000])

In [ ]:
plt.hist(col_size)

In [ ]:
plt.hist(row_size[col_size<1000])

In [ ]:
len(data.train_ds), len(data.test_ds)

In [ ]:
len(data.classes), data.classes[:5]

## Initial Model

In [ ]:
# FIXME: change num_workers to 8 to match num GPUs?
def get_data(arch, image_size, batch_size):
    tfms = tfms_from_model(arch, image_size, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test', num_workers=4,
                                    val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=batch_size)
    return data if image_size > 300 else data.resize(340, 'tmp')

### Precompute

In [ ]:
data = get_data(arch, image_size, batch_size)

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
learn.fit(1e-2, 5)

### Augment

In [ ]:
learn.precompute = False

In [ ]:
learn.fit(1e-2, 5, cycle_len=1)

In [ ]:
learn.save('224_pre')

In [ ]:
learn.load('224_pre')

### Increase Size

In [ ]:
learn.set_data(get_data(arch, 299, batch_size))
learn.freeze()

In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

### Test Time Augmentation

In [ ]:
log_preds,y = learn.TTA()
probs = np.exp(log_preds)
accuracy(log_preds,y), metrics.log_loss(y, probs)

In [ ]:
learn.save('299.pre')

In [ ]:
learn.load('299_pre')

In [ ]:
learn.fit(1e-2, 1, cycle_len=2)